# Precious Metals: Daily Spot Price Snapshot
A quick daily view of gold, silver, platinum, and palladium with performance, volatility, and a simple 12-month trend projection.


**Data source & cadence:** Alpha Vantage (daily), using `TIME_SERIES_DAILY` with `symbolUSD` and an `FX_DAILY` fallback, refreshed each morning with the `AV_API_KEY` secret. Timestamps below show the latest run (UTC).


# Precious Metal Spot Price Comparison (5 Years)

This notebook compares precious metal spot prices over the last five years and builds a simple forecast using interactive charts.

## Where to get spot price data

Common sources for spot price data include:

- **LBMA (London Bullion Market Association)**: Official daily gold and silver price benchmarks. Useful for authoritative spot pricing.
- **Metals-API**: Paid/free tiers with JSON API access for multiple metals (gold, silver, platinum, palladium).
- **Alpha Vantage**: Free tier provides precious metals data with API keys and rate limits.
- **Quandl/Nasdaq Data Link**: Offers LBMA and other datasets (free and paid).
- **Yahoo Finance**: Convenient access for analysis (e.g., `XAUUSD=X`, `XAGUSD=X`, `XPTUSD=X`, `XPDUSD=X`). While not an official benchmark, it's easy to use for exploratory analysis.

This notebook uses **Yahoo Finance** spot proxies for convenience.


In [ ]:
# If needed, install dependencies
# !pip install -q yfinance pandas plotly statsmodels


In [ ]:
import os
import time
import pandas as pd
import requests
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from datetime import datetime, timezone
from IPython.display import Markdown, display


In [ ]:
timestamp = datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M UTC')
display(Markdown(f'**Last updated:** {timestamp}'))


### Data download
Download the last five years of daily closes for each metal.


In [ ]:
tickers = {
    "Gold (XAUUSD)": "XAU",
    "Silver (XAGUSD)": "XAG",
    "Platinum (XPTUSD)": "XPT",
    "Palladium (XPDUSD)": "XPD",
}

api_key = os.environ.get("AV_API_KEY")
if not api_key:
    raise ValueError("Missing AV_API_KEY environment variable for Alpha Vantage.")

def fetch_series(symbol, label):
    attempts = [
        ("TIME_SERIES_DAILY", {"symbol": f"{symbol}USD"}, "Time Series (Daily)"),
        ("FX_DAILY", {"from_symbol": symbol, "to_symbol": "USD"}, "Time Series FX (Daily)"),
    ]
    for func, params, key in attempts:
        query = {"function": func, "apikey": api_key, **params, "outputsize": "full"}
        resp = requests.get("https://www.alphavantage.co/query", params=query, timeout=30)
        resp.raise_for_status()
        payload = resp.json()
        if key not in payload or not payload.get(key):
            note = payload.get("Note") or payload.get("Error Message")
            if func == attempts[-1][0]:
                return None, note or "No time series in response."
            time.sleep(12)
            continue
        rows = []
        for date_str, row in payload[key].items():
            close = row.get("4. close")
            if close is None:
                continue
            rows.append((pd.to_datetime(date_str), float(close)))
        if rows:
            series = pd.Series(dict(rows), name=label).sort_index()
            return series, None
        else:
            return None, "No close prices found."
    return None, "No data returned."

price_frames = []
skipped = []
for label, symbol in tickers.items():
    series, err = fetch_series(symbol, label)
    if series is None:
        skipped.append(f"{label} ({err})")
    else:
        price_frames.append(series)
    time.sleep(12)  # Respect AV rate limits (5 calls/min)

if not price_frames:
    raise ValueError("No price data returned from Alpha Vantage for any metal.")

prices = pd.concat(price_frames, axis=1).sort_index().ffill().dropna(how="all")
if prices.empty:
    raise ValueError("Price data empty after cleaning; check Alpha Vantage availability.")
prices.index.name = "Date"
prices.tail()


In [ ]:
normalized = prices / prices.iloc[0] * 100
normalized.head()


In [ ]:
pct_change = prices.pct_change().dropna() * 100
pct_change.head()


### Spot price history
See how absolute prices have moved over time.


In [ ]:
fig = px.line(
    prices.reset_index(),
    x="Date",
    y=prices.columns,
    title="Spot Price Comparison (Last 5 Years)",
    labels={"value": "USD per troy ounce", "Date": "Date"},
)
fig.update_layout(legend_title_text="Metal")
fig.show()


### Performance since start
Each series is rebased to 100 on day one to compare relative performance.


In [ ]:
fig = px.line(
    normalized.reset_index(),
    x="Date",
    y=normalized.columns,
    title="Normalized Performance (100 = Start of Period)",
    labels={"value": "Index (100 = start)", "Date": "Date"},
)
fig.update_layout(legend_title_text="Metal")
fig.show()


### Day-to-day volatility
Daily percent change highlights short-term swings and risk.


In [ ]:
fig = px.line(
    pct_change.reset_index(),
    x="Date",
    y=pct_change.columns,
    title="Daily Percent Change (Last 5 Years)",
    labels={"value": "Percent change (%)", "Date": "Date"},
)
fig.update_layout(legend_title_text="Metal")
fig.show()


## Simple Forecast (12 Months)

We use a basic Holt-Winters exponential smoothing model on monthly averages to produce a simple, transparent forecast.


### 12-month outlook
Monthly averages feed a simple Holt-Winters model to sketch a near-term trend. Treat this as directional only.


In [ ]:
monthly = prices.resample("M").mean().dropna()
forecast_horizon = 12

forecast_frames = []
for metal in monthly.columns:
    model = ExponentialSmoothing(
        monthly[metal],
        trend="add",
        seasonal=None,
        initialization_method="estimated",
    ).fit()
    forecast = model.forecast(forecast_horizon)
    forecast.name = metal
    forecast_frames.append(forecast)

forecast_df = pd.concat(forecast_frames, axis=1)
forecast_df.tail()


In [ ]:
fig = go.Figure()
for metal in monthly.columns:
    fig.add_trace(
        go.Scatter(
            x=monthly.index,
            y=monthly[metal],
            name=f"{metal} (history)",
        )
    )
    fig.add_trace(
        go.Scatter(
            x=forecast_df.index,
            y=forecast_df[metal],
            name=f"{metal} (forecast)",
            line=dict(dash="dash"),
        )
    )

fig.update_layout(
    title="12-Month Forecast (Holt-Winters, Monthly Avg)",
    xaxis_title="Date",
    yaxis_title="USD per troy ounce",
    legend_title_text="Series",
)
fig.show()
